In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator

import pymongo

In [2]:
load_dotenv()

True

#### Paris: Palais Garnier Sao Paulo: Sala Sao Paulo

In [37]:
paris_ld = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

In [23]:
melbourne_cbd = {'type': 'Point', 'coordinates': ['-37.81227175879301','144.96262186884945']}

In [24]:
sao_paulo_zc = {'type': 'Point', 'coordinates': ['-23.533215960575276','-46.639408758375076']}

In [26]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [ ]:
#def func(url, city, venue):
    cathegories = []
    for c in cathegories:
        

In [27]:
datap = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'vegan')

In [44]:
datapa = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'airport')

In [50]:
datapg = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'gare')

In [38]:
datapsb = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'starbucks')

In [56]:
datapv = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'veterinaire')

In [60]:
datapb = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'basketball')

In [65]:
datasc = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'ecole')

In [98]:
datap_= datap['response']['venues']

In [99]:
len(datasc_)

50

In [107]:
datap_[1]

{'id': '558038c8498e11d4ab3104eb',
 'name': 'Véganie',
 'contact': {},
 'location': {'address': '96 rue Quincampoix',
  'lat': 48.86290042062885,
  'lng': 2.3515370674431324,
  'labeledLatLngs': [{'label': 'display',
    'lat': 48.86290042062885,
    'lng': 2.3515370674431324}],
  'distance': 8555,
  'postalCode': '75003',
  'cc': 'FR',
  'city': 'Paris',
  'state': 'Île-de-France',
  'country': 'France',
  'formattedAddress': ['96 rue Quincampoix', '75003 Paris', 'France']},
 'categories': [{'id': '4bf58dd8d48988d10c951735',
   'name': 'Cosmetics Shop',
   'pluralName': 'Cosmetics Shops',
   'shortName': 'Cosmetics',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/beauty_cosmetic_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'hereNow': {'count': 0, 'summary'

In [73]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

In [74]:
def type_point(gps):
    return {"type": "Point", "coordinates": gps}

#### HACER FUNCION PARA GENERAR UN JSON POR CADA DATA agregar Tipo (vegan, starb, etc)

In [114]:
map_name = ['name']
map_categories = ['categories']
map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']
gastr = []
for dic in datap_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['categories']= getFromDict(dic,map_categories)
    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

In [75]:
map_name = ['name']
map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']
gastr = []
for dic in datapsb_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

#### HACER FUNCION PARA GENERAR LOS JSON

In [115]:
#def jsongenerator(name):
with open('vegans.json', 'w') as f:
    json.dump(gastr, f)

In [76]:
with open('starbucks.json', 'w') as f:
    json.dump(gastr, f)

#### llamada a la collection

In [127]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [128]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [129]:
g = db.get_collection("gastronomy")

In [130]:
g.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'city', 'latitude', 'longitude', 'location'])

In [122]:
query = {"location": {"$near": {"$geometry": paris_ld, "$minDistance": 0 ,"$maxDistance": 1000}}}

In [ ]:
#{'categories': [{'name': 'Restaurants'}]}

In [172]:
cond1 = {'categories': [0]}

In [173]:
cond2 = {"location": {"$near": {"$geometry": paris_ld, "$minDistance": 0 ,"$maxDistance": 10000}}}

In [174]:
query2 = {'$and': [cond1, cond2]}

In [175]:
list(g.find(query2))

[]